In [1]:
import re
from thefuzz import process
from collections import Counter
from pybliometrics.scopus import ScopusSearch, SerialTitle

re_art  = re.compile(r"@article\s\{.*\n(\t.*\n)+^\}", re.MULTILINE)
re_proc = re.compile(r"@proceedings\s\{.*\n(\t.*\n)+^\}", re.MULTILINE)
articles = ""
proceedings = ""
book = ""
with open("ieor-article.bib", 'r') as f:
    articles = f.read()

with open("ieor-proceedings.bib", 'r') as f:
    proceedings = f.read()

# with open("ieor-book.bib", 'r') as f:
#     books = f.read()  

/usr/local/lib/python3.11/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Creating config file at /Users/anand.kumar/.config/pybliometrics.cfg with default paths...
Configuration file successfully created at /Users/anand.kumar/.config/pybliometrics.cfg
For details see https://pybliometrics.rtfd.io/en/stable/configuration.html.


In [2]:
import pandas as pd

pub = pd.DataFrame(columns=["id", "title", "journal", "year", "authors", "kind", "extra"])

def add(pub, kind, regex, string):
    recomp = re.compile(regex, re.MULTILINE)
    for ar in re.finditer(recomp, string):
        art = ar.group(0)
        title = re.findall(r'\ttitle\s*=\s*\{(.*)\}', art); title = title[0] if len(title)>0 else pd.NA
        journal = re.findall(r'\tjournal\s*=\s*\{(.*)\}',art); journal = journal[0] if len(journal)>0 else pd.NA
        auth  = re.findall(r'\tauthor\s*=\s*\{(.*)\}',art); auth = auth[0] if len(auth)>0 else pd.NA
        year = re.findall(r'\tyear\s*=\s*\{(.*)\}',art); year = year[0] if len(year)>0 else pd.NA
        extra = art
        pub.loc[len(pub.index)] = [len(pub.index), title, journal, year, auth, kind, extra]
    return pub

pub = add(pub, "article", r"@article\s\{.*\n(\t.*\n)+^\}", articles)
pub = add(pub, "proceedings", r"@proceedings\s\{.*\n(\t.*\n)+^\}",  proceedings)
# pub = add(pub, "inbook", r"@inbook\s\{.*\n(\t.*\n)+^\}",  books)
# pub = add(pub, "booklet", r"@booklet\s\{.*\n(\t.*\n)+^\}",  books)
# pub = add(pub, "book", r"@book\s\{.*\n(\t.*\n)+^\}",  books)

In [3]:
# process authors 
pub['auth'] = pub['authors'].apply(lambda x: [y.strip() for y in x.split(" and ")])

In [4]:
# Get Duplicate titles (7 duplicates in last 5 years)
print(pub[pub.duplicated(subset=['title'])][['title', 'year']])
pub = pub.drop_duplicates(subset=['title'])

                                                 title  year
31       Multi-modal supply chain distribution problem  2021
42   Who is willing to pay for solar lamps in rural...  2021
54   Minotaur: a mixed-integer nonlinear optimizati...  2020
96   Optimizing large scale upgradations involving ...  2018
119  Local Polyhedrality of Integer Hulls of Strict...  2016
295                      Dynamic Bus Dispatch Policies  2020
296  Financial replicator dynamics: emergence of sy...  2020
326  Does involvement of local community ensure sus...  2018
423  Optimal auctions for multi-unit procurement wi...  2007
436  Development and benchmarking of an epoch time ...  2005
477  Optimizing loading policies at a bottleneck fa...  1996
484  Simulation for Modeling and Analysis of Burn C...   NaN


In [5]:
# # Finding title duplicates using fuzzy matching 
# for i in range(len(pub.title)):
#     matches = process.extractOne(pub.title.iloc[i], pub.title.iloc[i+1:], score_cutoff=90)
#     if matches:
#         print(pub.title.iloc[i], matches)


# # TILL 2018
#     # Multi-player age-of-information bandits: A trekking approach ('Multi-player bandits: A trekking approach', 95, 33)
#     # Submodular function minimization and polarity ('Submodular Function Minimization and Polarity', 100, 59)
#     # The effect of rolling stock standardization on rake utilization and punctuality of services ('Effect of standardization on rake utilization in rolling stock', 93, 71)
#     # User Response Based Recommendations ('User response based recommendations: A local angle approach', 90, 337)
#     # Challenges in Net Neutrality Violation Detection: A Case Study of Wehe Tool and Improvements ('Challenges in Net Neutrality Violation Detection: A Case Study of Wehe Tool', 95, 282)
#     # Controlling packet drops to improve freshness of information ('Controlling Packet Drops to Improve Freshness of information', 100, 294)

# # All FUZZY MATCHES
#     # Multi-player age-of-information bandits: A trekking approach ('Multi-player bandits: A trekking approach', 95, 33)
#     # Minotaur: a mixed-integer nonlinear optimization toolkit ('Mixed-integer nonlinear optimization', 90, 152)
#     # The effect of rolling stock standardization on rake utilization and punctuality of services ('Effect of standardization on rake utilization in rolling stock', 93, 70)
#     # Opportunistic Schedulers: Price for Fairness? ('Price of Fairness for Opportunistic and Priority Schedulers', 95, 359)
#     # User Response Based Recommendations ('User response based recommendations: A local angle approach', 90, 335)
#     # Hybrid approach using simulation-based optimisation for Job Shop Scheduling Problems ('Simulation and Optimisation Based Approach for Job Shop Scheduling Problems', 92, 348)
#     # Optimal Static Hedging of Currency Risk Using FX Forwards ('Optimal Static Hedging of Currency Risk Using FX Options and FX Forwards contracts', 95, 144)
#     # Modelling Disruptions and Resolving Conflicts Optimally in a Railway Schedule ('Modelling disruptions and resolving conflicts optimally in a railway schedule', 100, 169)
#     # A Computational study of a stochastic game based pollution tax model ('A stochastic game based model for pollution tax', 91, 410)
#     # Evolutionary Stability Against Multiple Mutations ('Spatial Games and Evolutionary Stability against Multiple Mutations', 95, 177)
#     # Calculation of constructible set of valid configurations for product variety management ('Product Variety Management', 90, 432)
#     # Scheduling in stochastic job shops using hybrid simulation and optimization approaches ('Stochastic Job Shop Scheduling using Hybrid Simulation and Optimization Approaches', 93, 399)
#     # Pricing surplus server capacity for mean waiting time sensitive customers ('A proof of the conjecture in Pricing surplus server capacity for mean waiting time sensitive customers', 95, 318)
#     # Hybrid system dynamic - Discrete event simulation-based architecture for hierarchical production planning ('Hierarchical production planning using a hybrid system dynamic-discrete event simulation architecture', 91, 446)
#     # DiffServ node with join minimum cost queue policy and multiclass traffic ('DiffServ node with join minimum cost queue policy: Analysis with multiclass traffic', 92, 459)
#     # A probabilistic approach to second order variational inequalities with bilateral constraints ('A probabilistic approach to viscosity solution of Second Order Variational Inequalities with Bilateral Constraints', 95, 244)
#     # Challenges in Net Neutrality Violation Detection: A Case Study of Wehe Tool and Improvements ('Challenges in Net Neutrality Violation Detection: A Case Study of Wehe Tool', 95, 281)
#     # Asymmetric Information Acquisition Games ('Acquisition  Games with Partial-Asymmetric Information', 95, 306)
#     # Perishable inventory management and dynamic pricing using RFID technology ('Perishable Inventory Management and RFID Technology', 95, 449)

# # removed (505 - 2 = 503)
# # - Submodular function minimization and polarity 
# # - Controlling packet drops to improve freshness of information

# # undecided 
# # - User Response Based Recommendations - presented in comsnets, published in accompanying journal (both entries present)
# # - Challenges in Net Neutrality Violation Detection - Presented in 2021, 2022 comsnets?

In [6]:
# 19 books 
# 486 articles + conferences 

pub5 = pub[(pub.year.astype('Int16')>=2018) | (pd.isna(pub.year))]

print("number of articles and proceedings in last 5 years : %d + 5 books" % (pub5.shape[0],))

number of articles and proceedings in last 5 years : 167 + 5 books


In [7]:
# quick access dict author -> [list of paper id]
authorDict = {}
for (id, row) in pub5['auth'].items():
    for a in row: 
        authorDict[a] = authorDict.get(a, [])+[id]

In [8]:
pd.options.display.max_columns = 250 #Changes the number of columns diplayed (default is 20)
pd.options.display.max_rows = 500 #Changes the number of rows diplayed (default is 60)
pd.options.display.max_colwidth = 250 #Changes the number of characters in a cell so that the contents don't get truncated 
pub5

,id,title,journal,year,authors,kind,extra,auth
0,0,Almost exact risk budgeting with return forecasts for portfolio allocation,Operations Research Letters,2023,Avinash Bhardwaj and Manjesh Kumar Hanawal and Purushottam Parthasarathy,article,"@article {2843,\n\ttitle = {Almost exact risk budgeting with return forecasts for portfolio allocation},\n\tjournal = {Operations Research Letters},\n\tvolume = {51},\n\tyear = {2023},\n\tpages = {171-175},\n\tdoi = {https://doi.org/10.1016/j.orl...","[Avinash Bhardwaj, Manjesh Kumar Hanawal, Purushottam Parthasarathy]"
1,1,Approximate solutions to constrained risk-sensitive Markov decision processes,European Journal of Operational Research,2023,Uday M Kumar and Sanjay P Bhat and Veeraruna Kavitha and Nandyala Hemachandra,article,"@article {2846,\n\ttitle = {Approximate solutions to constrained risk-sensitive Markov decision processes},\n\tjournal = {European Journal of Operational Research},\n\tvolume = {310},\n\tyear = {2023},\n\tpages = {249-267},\n\tdoi = {https://doi....","[Uday M Kumar, Sanjay P Bhat, Veeraruna Kavitha, Nandyala Hemachandra]"
2,2,"Dynamics of solar energy entrepreneurship in rural Bihar, India",Energy for Sustainable Development,2023,Rajat Mishra and Sayli Shiradkar and Kelsey Werner and Tresa Maria and Praveen Kumar and Jayendran Venkateswaran and Chetan S. Solanki,article,"@article {2869,\n\ttitle = {Dynamics of solar energy entrepreneurship in rural Bihar, India},\n\tjournal = {Energy for Sustainable Development},\n\tvolume = {76},\n\tyear = {2023},\n\tmonth = {10/2023},\n\tpages = {101269},\n\tkeywords = {Causal ...","[Rajat Mishra, Sayli Shiradkar, Kelsey Werner, Tresa Maria, Praveen Kumar, Jayendran Venkateswaran, Chetan S. Solanki]"
3,3,An OR Approach for WORdle,Informs Transactions on Education,2023,Simran Lakhani and Mustafa Vora and Ashutosh Mahajan,article,"@article {2831,\n\ttitle = {An OR Approach for WORdle},\n\tjournal = {Informs Transactions on Education},\n\tyear = {2023},\n\tmonth = {4/2023},\n\tdoi = {https://doi.org/10.1287/ited.2023.0285},\n\tauthor = {Simran Lakhani and Mustafa Vora and A...","[Simran Lakhani, Mustafa Vora, Ashutosh Mahajan]"
4,4,Choice of Intraoperative Ultrasound adjuncts for Brain Tumor Surgery,BMC Med Inform Decis Mak,2022,Manoj Kumar and Santosh noronha and Narayan Rangaraj and Aliasgar Moiyadi and Prakash Shetty and Vikas kumar Singh,article,"@article {2643,\n\ttitle = {Choice of Intraoperative Ultrasound adjuncts for Brain Tumor Surgery},\n\tjournal = {BMC Med Inform Decis Mak},\n\tvolume = {22},\n\tyear = {2022},\n\tmonth = {11/2022},\n\ttype = {Research},\n\tdoi = {https://doi.org/...","[Manoj Kumar, Santosh noronha, Narayan Rangaraj, Aliasgar Moiyadi, Prakash Shetty, Vikas kumar Singh]"
5,5,Data-Driven treatment policies for Dialysis patients,International Conference on Operations Research - OR 2022,2022,"Manoj Kumar and Usulumarty, Deepa and Ravi, B and Billa, Viswanath and Narayan Rangaraj and Santosh noronha",article,"@article {2658,\n\ttitle = {Data-Driven treatment policies for Dialysis patients},\n\tjournal = {International Conference on Operations Research - OR 2022},\n\tyear = {2022},\n\taddress = {Karlsruhe, Germany},\n\tauthor = {Manoj Kumar and Usuluma...","[Manoj Kumar, Usulumarty, Deepa, Ravi, B, Billa, Viswanath, Narayan Rangaraj, Santosh noronha]"
6,6,Evading Malware Analysis Using Reverse Execution,2022 14th International Conference on COMmunication Systems \& NETworkS (COMSNETS),2022,"Mishra, Adhokshaj and Roy, Animesh and Manjesh Kumar Hanawal",article,"@article {2655,\n\ttitle = {Evading Malware Analysis Using Reverse Execution},\n\tjournal = {2022 14th International Conference on COMmunication Systems \& NETworkS (COMSNETS)},\n\tyear = {2022},\n\tpublisher = {IEEE},\n\tauthor = {Mishra, Adhoks...","[Mishra, Adhokshaj, Roy, Animesh, Manjesh Kumar Hanawal]"
7,7,Exact and heuristic approaches for the Railway crew scheduling,6th AIROYoung Workshop,2022,Akshat Bansal and Anoop K. P. and Narayan Rangaraj,a

### Accesing SCOPUS for metrics

In [91]:
# for id, row in pub5.iterrows():
#     try: 
#         pub5.loc[id, 'scopus_data'] = ScopusSearch(row.title + " - " + row.auth[0])
#     except:
#         print("id: %d \tquery : %s\n"%(id, row.title + " - " + row.auth[0]))


# pub5.to_pickle('ieor5Y-scopus')

id: 24 	query : A journey towards Energy Swaraj (independence): Energy by locals for locals - Chetan S. Solanki

id: 99 	query : Perspectives on Operations Research (IEOR Seminar Series) - Jayendran Venkateswaran

id: 269 	query : Single-out fake posts: participation game and its design. (accepted and presented at American Control Conference (ACC) 2023) - Agarwal, Khushboo

id: 280 	query : Stability Analysis of Emission-based Production and Inventory Control Systems (EPICS) - Rishav Deval



In [95]:
pub5 = pd.read_pickle('ieor5Y-scopus')

# # get first match, 
# # TODO: VERIFY LATER FOR CORRECTNESS 
# pub5["scopus_match"] = pub5.scopus_data.apply(lambda x: '' if x=='' else x.results[0]._asdict() if x.get_results_size()>0 else "0")

# # Get ISSN for matched results 
# pub5['issn'] = pub5.scopus_match.apply(lambda x: x['issn'] if isinstance(x, dict) else pd.NA)

# pub5.to_pickle('ieor5Y-scopus')

In [130]:
pub5 = pd.read_pickle('ieor5Y-scopus')

pub5["source details"] = pub5.issn.apply(lambda x: SerialTitle(x) if not pd.isna(x) else pd.NA)

# TODO: maybe implement row wise instead of apply and handle errors on case by case basis

Scopus404Error: The resource specified cannot be found.

In [131]:
pub5.head()

,id,title,journal,year,authors,kind,extra,auth,scopus_data,scopus_match,issn
0,0,Almost exact risk budgeting with return forecasts for portfolio allocation,Operations Research Letters,2023,Avinash Bhardwaj and Manjesh Kumar Hanawal and Purushottam Parthasarathy,article,"@article {2843,\n\ttitle = {Almost exact risk budgeting with return forecasts for portfolio allocation},\n\tjournal = {Operations Research Letters},\n\tvolume = {51},\n\tyear = {2023},\n\tpages = {171-175},\n\tdoi = {https://doi.org/10.1016/j.orl...","[Avinash Bhardwaj, Manjesh Kumar Hanawal, Purushottam Parthasarathy]",Search 'Almost exact risk budgeting with return forecasts for portfolio allocation - Avinash Bhardwaj' yielded 1 document as of 2023-08-21:\n 2-s2.0-85150336279,"{'eid': '2-s2.0-85150336279', 'doi': '10.1016/j.orl.2023.02.002', 'pii': 'S0167637723000263', 'pubmed_id': None, 'title': 'Almost exact risk budgeting with return forecasts for portfolio allocation', 'subtype': 'ar', 'subtypeDescription': 'Articl...",01676377
1,1,Approximate solutions to constrained risk-sensitive Markov decision processes,European Journal of Operational Research,2023,Uday M Kumar and Sanjay P Bhat and Veeraruna Kavitha and Nandyala Hemachandra,article,"@article {2846,\n\ttitle = {Approximate solutions to constrained risk-sensitive Markov decision processes},\n\tjournal = {European Journal of Operational Research},\n\tvolume = {310},\n\tyear = {2023},\n\tpages = {249-267},\n\tdoi = {https://doi....","[Uday M Kumar, Sanjay P Bhat, Veeraruna Kavitha, Nandyala Hemachandra]",Search 'Approximate solutions to constrained risk-sensitive Markov decision processes - Uday M Kumar' yielded 2 documents as of 2023-08-22:\n 2-s2.0-85151031892\n 2-s2.0-85161560717,"{'eid': '2-s2.0-85151031892', 'doi': '10.1016/j.ejor.2023.02.039', 'pii': 'S0377221723001893', 'pubmed_id': None, 'title': 'Approximate solutions to constrained risk-sensitive Markov decision processes', 'subtype': 'ar', 'subtypeDescription': 'Ar...",03772217
2,2,"Dynamics of solar energy entrepreneurship in rural Bihar, India",Energy for Sustainable Development,2023,Rajat Mishra and Sayli Shiradkar and Kelsey Werner and Tresa Maria and Praveen Kumar and Jayendran Venkateswaran and Chetan S. Solanki,article,"@article {2869,\n\ttitle = {Dynamics of solar energy entrepreneurship in rural Bihar, India},\n\tjournal = {Energy for Sustainable Development},\n\tvolume = {76},\n\tyear = {2023},\n\tmonth = {10/2023},\n\tpages = {101269},\n\tkeywords = {Causal ...","[Rajat Mishra, Sayli Shiradkar, Kelsey Werner, Tresa Maria, Praveen Kumar, Jayendran Venkateswaran, Chetan S. Solanki]","Search 'Dynamics of solar energy entrepreneurship in rural Bihar, India - Rajat Mishra' yielded 0 documents as of 2023-08-22",0,<NA>
3,3,An OR Approach for WORdle,Informs Transactions on Education,2023,Simran Lakhani and Mustafa Vora and Ashutosh Mahajan,article,"@article {2831,\n\ttitle = {An OR Approach for WORdle},\n\tjournal = {Informs Transactions on Education},\n\tyear = {2023},\n\tmonth = {4/2023},\n\tdoi = {https://doi.org/10.1287/ited.2023.0285},\n\tauthor = {Simran Lakhani and Mustafa Vora and A...","[Simran Lakhani, Mustafa Vora, Ashutosh Mahajan]",Search 'An OR Approach for WORdle - Simran Lakhani' yielded 0 documents as of 2023-08-22,0,<NA>
4,4,Choice of Intraoperative Ultrasound adjuncts for Brain Tumor Surgery,BMC Med Inform Decis Mak,2022,Manoj Kumar and Santosh noronha and Narayan Rangaraj and Aliasgar Moiyadi and Prakash Shetty and Vikas kumar Singh,article,"@article {2643,\n\ttitle = {Choice of Intraoperative Ultrasound adjuncts for Brain Tumor Surgery},\n\tjournal = {BMC Med Inform Decis Mak},\n\tvolume = {22},\n\tyear = {2022},\n\tmonth = {11/2022},\n\ttype = {Research},\n\tdoi = {https://doi.org/...","[Manoj Kumar, Santosh noronha, Narayan Rangaraj, Aliasgar Moiyadi, Prakash Shetty, Vikas kumar Singh]",Search 'Choice of Intraoperative Ultrasound adjuncts for Brain Tumor Surgery - Manoj Kumar' yielded 6 documents as of 2023-08-22:\n 

In [122]:
SerialTitle()

True

In [43]:
# respub.citescoreyearinfolist
# respub.sniplist
# respub.sjrlist

[(2022, 0.583)]

### Data extraction

In [14]:
print(Counter(pub5[pub5.kind=="proceedings"].year))
print(Counter(pub5[pub5.kind=="article"].year))

Counter({'2019': 16, '2020': 13, '2018': 13, '2021': 10, '2022': 9, '2023': 4})
Counter({'2021': 26, '2019': 22, '2018': 20, '2020': 17, '2022': 11, '2023': 4, nan: 2})


In [10]:
# find co
def coauthors(author, df = pub5):
    coauth = []
    for id, row in df.iterrows():
        if author in row['auth']: 
            coauth += row['auth']
    return Counter(coauth)

coauthors('Ashutosh Mahajan')

Counter({'Ashutosh Mahajan': 8,
         'H. Rahman': 2,
         'Simran Lakhani': 1,
         'Mustafa Vora': 1,
         'Hussain A. Kharodawala': 1,
         'Jayan Moorkanat': 1,
         'Sven Leyffer': 1,
         'Jeff Linderoth': 1,
         'James Luedtke': 1,
         'Todd Munson': 1,
         'Ladage, Anurag': 1,
         'Baatar, Davaatseren': 1,
         'Krishnamoorthy, Mohan': 1,
         'Akondi Praneeth': 1,
         'Jayendran Venkateswaran': 1,
         'N. Hooda': 1,
         'Om Damani': 1})

In [11]:
print(Counter(pub5.journal.to_list()))

Counter({'arXiv-preprint': 8, nan: 5, '2022 14th International Conference on COMmunication Systems \\& NETworkS (COMSNETS)': 4, 'XXIII Annual International Conference of the Society of Operations Management': 3, 'European Journal of Operational Research': 2, 'Energy for Sustainable Development': 2, 'Dynamic Games and Applications': 2, 'OPSEARCH': 2, 'Performance Evaluation': 2, 'International Journal of Game Theory': 2, 'Advances in Neural Information Processing Systems': 2, '38th International Conference of the System Dynamics Society': 2, 'Annals of Operations Research': 2, 'Computer Communications': 2, 'SOM-IMTH Summer School in Operations and Supply Chain Management Research': 2, '29th European Conference on Operational Research (EURO 2018)': 2, 'American Control Conference (ACC)': 2, 'IEEE International Conference on Industrial Engineering and Engineering Management (IEEM)': 2, 'International Conference on Network Games, Control and Optimization': 2, 'Neural Information Processing

In [14]:
# Check for duplicate authors
auth = list(authorDict.keys())

for i in range(len(auth)):
    matches = process.extractBests(auth[i], auth[i+1:], score_cutoff=80)
    print(auth[i], matches)

Avinash Bhardwaj []
Manjesh Kumar Hanawal [('Uday M Kumar', 86), ('Praveen Kumar', 86), ('Manoj Kumar', 86), ('rajesh Kumar', 86), ('Sandeep Kumar', 86)]
Purushottam Parthasarathy []
Uday M Kumar [("Gyetvai, M{\\'a}rton", 86), ('R. Kumar', 86), ('M Venkateswararao Koppisetti', 86)]
Sanjay P Bhat [('P Mallikarjuna Reddy', 86)]
Veeraruna Kavitha []
Nandyala Hemachandra []
Rajat Mishra []
Sayli Shiradkar []
Kelsey Werner []
Tresa Maria []
Praveen Kumar [('R. Kumar', 86)]
Jayendran Venkateswaran []
Chetan S. Solanki [('S. Kapsikar', 86), ('S Shekhar', 86)]
Simran Lakhani []
Mustafa Vora []
Ashutosh Mahajan []
Manoj Kumar [('Vikas kumar Singh', 86), ('Anil Kumar Sharma', 86), ('Raman Kumar Sinha', 86)]
Santosh noronha []
Narayan Rangaraj []
Aliasgar Moiyadi []
Prakash Shetty [('Rudrabhotla Sri Prakash', 86)]
Vikas kumar Singh [('R. Kumar', 86), ('Rohit Kumar', 86)]
Usulumarty, Deepa []
Ravi, B [('Ravi Kant Rai', 86), ('Sunoj, Raghavan B.', 86)]
Billa, Viswanath []
Mishra, Adhokshaj []
Roy, 

In [66]:
pub5.to_csv('last5years.csv')

In [15]:
from pprint import pprint as pprint
pprint(authorDict.keys())

dict_keys(['Avinash Bhardwaj', 'Manjesh Kumar Hanawal', 'Purushottam Parthasarathy', 'Uday M Kumar', 'Sanjay P Bhat', 'Veeraruna Kavitha', 'Nandyala Hemachandra', 'Rajat Mishra', 'Sayli Shiradkar', 'Kelsey Werner', 'Tresa Maria', 'Praveen Kumar', 'Jayendran Venkateswaran', 'Chetan S. Solanki', 'Simran Lakhani', 'Mustafa Vora', 'Ashutosh Mahajan', 'Manoj Kumar', 'Santosh noronha', 'Narayan Rangaraj', 'Aliasgar Moiyadi', 'Prakash Shetty', 'Vikas kumar Singh', 'Usulumarty, Deepa', 'Ravi, B', 'Billa, Viswanath', 'Mishra, Adhokshaj', 'Roy, Animesh', 'Akshat Bansal', 'Anoop K. P.', 'Sumit J. Darak', 'Prashant Trivedi', 'Hussain A. Kharodawala', 'Jayan Moorkanat', 'Gudwani, Hitesh', 'Sharayu Moharir', 'Swapnesh Subramanian', 'Shrirang Bichu', 'Kothari, Jatin', 'rajesh Kumar', 'Kalpana Mehta', 'Tukaram Jamale', 'Nikhil Bhasin', 'Sayali Thakare', 'Smriti Sinha', 'Geeta Sheth', 'Verma, Utkarsh', 'Verma, Arun', 'Arun Rajkumar', 'Raman Sankaran', 'Shiksha Singhal', 'Vaze, Rahul', 'Agarwal, Khushbo